In [33]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers




In [34]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [35]:
extracted_data = load_pdf("data/")

In [36]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [37]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [38]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings_model = download_hugging_face_embeddings()


In [39]:
embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [40]:
query_result = embeddings_model.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [41]:

from pinecone import Pinecone

# Initialize Pinecone client
api_key = "76005231-c9e4-4584-8eec-1ab3080a8019"
pc = Pinecone(api_key=api_key, environment="aws", region="us-east-1", service="chatbot-frq4voi")

index_name = "chatbot"

In [63]:
def generate_embeddings(text_chunks, embeddings_model):
    embeddings_list = []
    for chunk in text_chunks:
        # Generate embedding for each chunk of text
        embedding = embeddings_model.encode(str(chunk))
        embeddings_list.append(embedding)  # Assuming only one embedding per chunk
    return embeddings_list


In [64]:
embeddings_list = generate_embeddings(text_chunks, embeddings_model)

AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'encode'

In [65]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(str(text_chunks))
print(embeddings)

[-3.17879394e-02  4.90619689e-02 -5.28125949e-02  2.07640603e-02
  8.27199221e-03 -8.12541507e-03 -6.38942719e-02  6.47773668e-02
 -1.54679660e-02  3.33526842e-02  3.99441831e-02 -1.08290920e-02
  4.01090868e-02 -7.22186863e-02 -5.50658815e-02  5.25086559e-02
 -1.30023316e-01 -4.68452945e-02 -8.10969323e-02 -6.27822354e-02
 -9.31243121e-04  1.05626449e-01  5.93985505e-02 -1.88697055e-02
  3.56282480e-02 -3.38086300e-03 -7.48195499e-02  8.45949166e-03
 -6.21308461e-02 -5.69668636e-02  5.06398715e-02  5.20430617e-02
  3.43506448e-02  5.72989807e-02  1.33494422e-01  6.39560148e-02
 -1.75358038e-02  2.09317333e-03  5.53194135e-02  1.01458989e-01
 -2.35288423e-02 -8.22416767e-02 -3.89629379e-02 -2.78309509e-02
 -2.17930619e-02 -6.97924048e-02 -7.33964071e-02 -6.68306872e-02
  8.18725675e-02  7.35397413e-02 -1.58618137e-01 -2.61132047e-02
 -6.47537336e-02  4.72263582e-02  2.75570378e-02 -7.23998819e-04
 -2.21583694e-02 -9.31661651e-02 -1.00332916e-01 -7.61337131e-02
 -2.88476534e-02  1.12586

In [67]:
len(embeddings)

384